# Taller 5 (continuación) - FFT 

#### Comparación de DFT (transformada discreta de Fourier) con FFT (transformada rápida) algoritmo propio en python VS implementación con numpy y scypy

### Referencias

* https://pythonnumericalmethods.berkeley.edu/notebooks/chapter24.03-Fast-Fourier-Transform.html

* https://pythonnumericalmethods.berkeley.edu/notebooks/chapter24.04-FFT-in-Python.html

* https://towardsdatascience.com/fast-fourier-transform-937926e591cb

* https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/

* https://es.wikipedia.org/wiki/Transformada_r%C3%A1pida_de_Fourier

* https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm


In [3]:
import numpy as np 

In [53]:
def get_data(N):
    data=np.random.random(N)
    return data

"""
def get_circular_terms(N):
    terms = np.exp(-1j*2*np.pi*np.arange(N)/N)
    return terms
"""
    
    
#Transformada discreta de Fourier; eficiencia O(N^{2})
def dft(data): 
    N = data.shape[0]
    n= np.arange(N)
    k= n.reshape((N,1))
    M = np.exp(-1j*2*np.pi*k*n/N)
    
    return np.dot(M,data)

#Transformada rápida Fourier: Eficiencia O(Nlog_{2}(N))

def fft(data):   
    """
    data: np.array  
    data as 1D array
    return discrete fourier transform of data
    """

    # len of data
    N = data.shape[0]
    # Durante la sesión de taller no escribí esta linea por lo que no había "N". 
    #Ese era el error que marcaba.  
    
    assert N%2 == 0, 'len of data: {} must be a power of 2'.format(N)
    
    if N <= 2: 
        return dft(data)
    else:
        data_even = fft(data[::2])
        data_odd = fft(data[1::2])
        #terms=get_circular_terms(N)
        """
        Llamar a otra función dentro de esta función alenta el algoritmo
        mejor llamar los términos "circulares directamente"...
        """
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        
        return np.concatenate(
            [
                data_even+terms[:int(N/2)]*data_odd,
                data_even+terms[int(N/2):]*data_odd
                
            ])

N=1024
x=get_data(N)
print("DATA: ", x)

DFT = dft(x)
FFT = fft(x)
FFTnp = np.fft.fft(x)
#print('DFT: ',DFT)
#print('FFT: ', FFT)
#print('FFTnp: ',FFTnp)

#Revisamos si nuestra implementación del algoritmo coincide con la de numpy
print((np.allclose(DFT,FFTnp),np.allclose(FFT,FFTnp)))
    


DATA:  [0.11724655 0.26173168 0.29546221 ... 0.05404687 0.9910709  0.61270735]
(True, True)


### Revisamos la rapidez de los algoritmos
Consideremos que la implementación de numpy y scipy están escritas en fortran y usan operaciones vectoriales en lugar de recursión lo que las hace mucho más rápidas como veremos. Revisar referencias para ver como se realiza la implemetación de la FFT con operaciones vectoriales.  

In [54]:
%timeit dft(x)
%timeit fft(x)
%timeit np.fft.fft(x)

130 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
32 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
86.2 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [52]:
import scipy.fftpack as sfft
%timeit sfft.fft(x)

26.5 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Vemos que para 1024 datos, la implementación de scipy resulta ser el algoritmo más eficiente.